## Actual Workings

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
!pip install transformers
!pip install pytorch_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 
     |████████████████████████████████| 163 kB 64.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 5.1 MB/s 
     |████████████████████████████████| 880 kB 44.0 MB/s 
     |████████████████████████████████| 1.3 MB 59.9 MB/s 
     |████████████████████████████████| 132 kB 63.6 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 9.2 MB 54.6 MB/s 
     |████████████████████████████████| 140 kB 64.7 MB/s 
     |████████████████████████████████| 127 kB 66.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2ffb89ffedd63d3abf0ca2397b5986631a2fc1806bd0f8c63c65f33f4f826745
  Stored

In [ ]:
import pandas as pd
import numpy as np

import torch

from transformers import BertTokenizer
from transformers import BertModel

from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
data_uri = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/data/allData-19-08-27.ta'

df = pd.read_csv(data_uri, header=None, names=['news'])
len(df)

1727888

In [ ]:
df['news'] = df['news'].str.strip()
df.head(), len(df)

(                                                news
 0  அமைச்சின் பணிகளை முன்னெடுப்பதற்கு கௌர அமைச்சர்...
 1  இவ்வமைச்சு இல 40  புத்கமுவ வீதி  இராஜகிரிய எனு...
 2  தேசிய மொழிகள் மற்றும் சமூக ஒருமைப்பாட்டு அமைச்...
 3  தேசிய மொழிகள்  சமூக ஒருமைப்பாடு தொடர்பான கொள்க...
 4  குறித்த தேசிய நோக்கங்களை எய்தும் பொருட்டு தேசி..., 1727888)

In [ ]:
## Load pretrained model/tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)
model = BertModel.from_pretrained('bert-base-multilingual-uncased', output_hidden_states=True)

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Put the model in "evaluation" mode,meaning feed-forward operation.
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(105879, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [ ]:
def get_vectors(news):
  encoding = tokenizer.encode(news,
                              add_special_tokens = True,
                              max_length = 512, 
                              truncation = True, 
                              padding = "max_length"
                            )
  token_text = tokenizer.convert_ids_to_tokens(encoding)
  indexed_tokens = tokenizer.convert_tokens_to_ids(token_text)

  # Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])

  # Run the text through BERT, get the output and collect all of the hidden states produced from all 12 layers.
  with torch.no_grad():
    outputs = model(tokens_tensor)
    
    # can use last hidden state as word embeddings
    last_hidden_state = outputs[0]
    word_embed_1 = last_hidden_state

    # Evaluating the model will return a different number of objects based on how it's  configured in the `from_pretrained` call earlier. 
    # In this case, becase we set `output_hidden_states = True`, the third item will be the hidden states from all layers. 
    # See the documentation for more details:https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]
    
    # initial embeddings can be taken from 0th layer of hidden states
    word_embed_2 = hidden_states[0]
    return word_embed_2

In [ ]:
df['vec'] = df['news'].progress_apply(get_vectors)

  0%|          | 0/1727888 [00:00<?, ?it/s]

In [ ]:
# base_url = r'D:\repo\R&D\deeplearning_news_clustering\data\vec.csv'
base_url = '/content/drive/MyDrive/Colab Notebooks/deeplearning_news_clustering/vectors/vec.csv'
df.to_csv(base_url, encoding='utf-8')